In [1]:
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import scipy
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Input, Flatten
from keras import utils
from sklearn.metrics import classification_report, confusion_matrix
from keras import backend as K

!python -m spacy download en_core_web_md
import en_core_web_md
uploaded = files.upload()

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=1e272a2bd3a55ca168d89c2f1383d97babf986f59706d137de820b72ad49f66e
  Stored in directory: /tmp/pip-ephem-wheel-cache-dabzx116/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


Saving tweets_data.csv to tweets_data.csv


In [2]:
df = pd.read_csv('tweets_data.csv')
df.head()

,tweet,hate
0,these girls are the equivalent of the irritati...,1
1,of course you were born in serbiayoure as fuck...,1
2,hopefully someone cooks drasko in the next ep ...,0
3,drasko they didnt cook half a bird you idiot mkr,0
4,so drasko just said he was impressed the girls...,0


In [3]:
# Load pre-trained spaCy model
nlp = en_core_web_md.load()

In [4]:
# Splitting dataset dataframe into X and Y
X,Y = df.iloc[:,0] , df.iloc[:,-1]
X = df.iloc[:, 0]
Y = df.iloc[:,-1]
X = np.array(X.apply(lambda x : nlp(x).vector))
X = [ list(i) for i in X]
X = np.asarray(X)

In [5]:
# Performance metrics
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

def train(X, Y):
    # Splitting dataset into 80-20 distribution between train-test 
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
    
    model = Sequential()
    # LSTM layer with 10 units and RELU activation
    model.add(LSTM(10, input_shape=(1, X_train.shape[2]), activation = 'relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m])
    history = model.fit(X_train, Y_train, epochs=3, batch_size=100, validation_split=0.1)
    y_pred = model.predict(X_test, batch_size=100)
    prediction = np.argmax(y_pred, axis=1)

In [6]:
train(X, Y)

Epoch 1/3
76/76 [==============================] - 1s 9ms/step - loss: 0.6932 - accuracy: 0.5471 - f1_m: 0.4122 - val_loss: 0.6932 - val_accuracy: 0.5824 - val_f1_m: 0.3881
Epoch 2/3
76/76 [==============================] - 0s 5ms/step - loss: 0.6931 - accuracy: 0.5728 - f1_m: 0.4121 - val_loss: 0.6931 - val_accuracy: 0.6097 - val_f1_m: 0.3881
Epoch 3/3
76/76 [==============================] - 0s 5ms/step - loss: 0.6931 - accuracy: 0.5706 - f1_m: 0.4121 - val_loss: 0.6931 - val_accuracy: 0.6251 - val_f1_m: 0.3881
